# Run Model Explanation with SageMaker Clarify (Post-Training)

## Using SageMake Processing Jobs

In [ ]:
import boto3
import sagemaker
import pandas as pd
import numpy as np

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

In [ ]:
%store -r training_job_name

In [ ]:
try:
    training_job_name
    print('[OK]')
except NameError:
    print('+++++++++++++++++++++++++++++++')
    print('[ERROR] Please run the notebooks in the previous TRAIN section before you continue.')
    print('+++++++++++++++++++++++++++++++')

In [ ]:
print(training_job_name)

# Review Data

In [ ]:
import pandas as pd

data = pd.read_json('./data-clarify/test_explainability.jsonl', lines=True)
data.head()

In [ ]:
data.shape

# Create Model

In [ ]:
import sagemaker

inference_image_uri = sagemaker.image_uris.retrieve(
    framework="tensorflow",
    region=region,
    version="2.3.1",
    py_version="py37",
    instance_type='ml.m5.4xlarge',
    image_scope="inference"
)
print(inference_image_uri)

In [ ]:
model_name = sess.create_model_from_job(
    training_job_name=training_job_name,
    image_uri=inference_image_uri
)

In [ ]:
print(model_name)

# Explaining Predictions with Amazon SageMaker Clarify

There are expanding business needs and legislative regulations that require explainations of _why_ a model mades the decision it did. SageMaker Clarify uses SHAP to explain the contribution that each input feature makes to the final decision.

## We need use a patch image to support JSONLines correctly for now
                         
`clarify_processor = clarify.SageMakerClarifyProcessor(...
        clarify_processor.image_uri = xxx`

In [ ]:
from sagemaker import clarify

clarify_processor = clarify.SageMakerClarifyProcessor(role=role,
                                                      instance_count=1,
                                                      instance_type='ml.c5.2xlarge',
                                                      sagemaker_session=sess)

# patch image 
clarify_processor.image_uri = "678264136642.dkr.ecr.us-east-1.amazonaws.com/sagemaker-clarify-processing:1.0_jsonlines_patch"

# Writing DataConfig and ModelConfig
A `DataConfig` object communicates some basic information about data I/O to Clarify. We specify where to find the input dataset, where to store the output, the target column (`label`), the header names, and the dataset type.

Similarly, the `ModelConfig` object communicates information about your trained model and `ModelPredictedLabelConfig` provides information on the format of your predictions.  

**Note**: To avoid additional traffic to your production models, SageMaker Clarify sets up and tears down a dedicated endpoint when processing. `ModelConfig` specifies your preferred instance type and instance count used to run your model on during Clarify's processing.

## Upload Data To S3

In [ ]:
path = './data-clarify/test_explainability.jsonl'

In [ ]:
!head -n 5 $path

In [ ]:
test_explainability_path_jsonlines_s3_uri = sess.upload_data(bucket=bucket, key_prefix=training_job_name, path=path)
test_explainability_path_jsonlines_s3_uri

In [ ]:
%store test_explainability_path_jsonlines_s3_uri

# Configure Clarify

In [ ]:
explainability_output_path = 's3://{}/clarify-explainability'.format(bucket)

explainability_data_config = clarify.DataConfig(s3_data_input_path=test_explainability_path_jsonlines_s3_uri,
                                s3_output_path=explainability_output_path,
                                headers=['review_body', 'product_category'],
                                features='features',
                                dataset_type='application/jsonlines')

In [ ]:
model_config = clarify.ModelConfig(model_name=model_name,
                                   instance_type='ml.m5.4xlarge',
                                   instance_count=1,
                                   content_type='application/jsonlines',
                                   accept_type='application/jsonlines',
                                   content_template='{"features":$features}')

# Run Clarify

In [ ]:
shap_config = clarify.SHAPConfig(baseline=[{"features":["ok", "Digital_Software"]}], # [data.iloc[0].values.tolist()],
                                 num_samples=5,
                                 agg_method='mean_abs')

In [ ]:
clarify_processor.run_explainability(model_config=model_config,
                                     model_scores='predicted_label',
                                     data_config=explainability_data_config,                                     
                                     explainability_config=shap_config,
                                     wait=False,
                                     logs=False)

In [ ]:
run_explainability_job_name = clarify_processor.latest_job.job_name
run_explainability_job_name

In [ ]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/processing-jobs/{}">Processing Job</a></b>'.format(region, run_explainability_job_name)))


In [ ]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a target="blank" href="https://console.aws.amazon.com/cloudwatch/home?region={}#logStream:group=/aws/sagemaker/ProcessingJobs;prefix={};streamFilter=typeLogStreamPrefix">CloudWatch Logs</a> After About 5 Minutes</b>'.format(region, run_explainability_job_name)))


In [ ]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a target="blank" href="https://s3.console.aws.amazon.com/s3/buckets/{}/{}/?region={}&tab=overview">S3 Output Data</a> After The Processing Job Has Completed</b>'.format(bucket, run_explainability_job_name, region)))


In [ ]:
running_processor = sagemaker.processing.ProcessingJob.from_processing_name(processing_job_name=run_explainability_job_name,
                                                                            sagemaker_session=sess)

processing_job_description = running_processor.describe()

print(processing_job_description)

In [ ]:
running_processor.wait(logs=False)

#### Viewing the Explainability Report
As with the bias report, you can view the explainability report in Studio under the experiments tab


<img src="img/explainability_detail.gif">

The Model Insights tab contains direct links to the report and model insights.

If you're not a Studio user yet, as with the Bias Report, you can access this report at the following S3 bucket.

# Download Report From S3

In [ ]:
!aws s3 ls $explainability_output_path/

In [ ]:
!aws s3 cp --recursive $explainability_output_path ./explainability_report/

In [ ]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a target="blank" href="./explainability_report/report.html">Explainability Report</a></b>'))


# Release Resources

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [ ]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}